## Social Security
####  Winter School on Smart Contracts
##### Sercan Ates, 15-917-024
2022-04-02


## Setup
See notebook 04.1, loading `algo_util.py`, the five accounts and the Purestake credentials
* Consider hiding this code

In [1]:
# Loading shared code and credentials
import sys, os
codepath = '..'+os.path.sep+'..'+os.path.sep+'sharedCode'
sys.path.append(codepath)
from algo_util import *
cred = load_credentials()
MyAlgo  = cred['MyAlgo']
Alice   = cred['Alice']
Bob     = cred['Bob']
Charlie = cred['Charlie']
Dina    = cred['Dina']

In [371]:
from algosdk import account, mnemonic, encoding,logic
from algosdk.v2client import algod
from algosdk.future import transaction
from algosdk.future.transaction import PaymentTxn
from algosdk.future.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn, LogicSig, LogicSigTransaction
import algosdk.error
import json
import base64
import datetime

In [372]:
from pyteal import *

In [373]:
# Initialize the algod client (Testnet or Mainnet)
algod_client = algod.AlgodClient(algod_token='', algod_address=cred['algod_test'], headers=cred['purestake_token'])
last_block = algod_client.status()["last-round"]
print(f"Last committed block is: {last_block}")

Last committed block is: 20613200


In [374]:
print(Alice['public'], "Alice")
print(Bob['public'], "Bob")
print(Charlie['public'], "Charlie")
print(Dina['public'], "Dina")
print(MyAlgo['public'], "MyAlgo")

V7BJQQBHMT5WHI5Z3NBUME4KXSRDZIP4BC6I5XMRBW63MYHUAGP5CQJXDI Alice
G6CQ6JO37JFGQBIW5IPP2OYWRN2XWGKQY4IXQ3SRNQC73ZTPCPLT6NNTPY Bob
DG2OLFAF5MK6VJP4L3C42FFBMHJ3JSUUKVQHVKBHDFLQA4O6OEKVKSRS3U Charlie
25UUED44Z2EUVCAFZUOC33LTJDE5C6ZI3UGDZPVPNH3NL3RSLLTCDW2WPA Dina
7ACAHM54YPV3IPQMSA7TDNVTDZZ52D3EZMX63Z64BQW3EGIEG7WYJWH5BQ MyAlgo


### Create Social Coin (ASA)

In [444]:
sp = algod_client.suggested_params()
token_supply = 10000                             # Token supply (big units)
token_decimals =  2                              # Digits after the comma
token_total = token_supply * 10**token_decimals  # Specify number of SMALLER unit ("cents")

token_name  = "Social Security Coin"                  # <----- YOUR NAME HERE
token_url   = ""                   # <----- CHANGE if you want to
token_unit  = "SSC"                              # Abbreviation, e.g. shown in Algorand Wallet app 

txn = AssetConfigTxn(
    sender=MyAlgo['public'],                   # Creator of the ASA
    sp=sp,                      
    total=token_total,                         # Total supply in SMALL unit
    decimals=token_decimals,                   # Decimals
    default_frozen=False,                      # Are tokens frozen by default?
    unit_name=token_unit,                      # Abbreviation     
    asset_name=token_name,                     # Name
    url=token_url,                             # URL
    manager=MyAlgo['public'],                   # Special roles (later more)
    reserve=MyAlgo['public'],                   # Special roles
    freeze=MyAlgo['public'],                      # Special roles
    clawback=MyAlgo['public']                     # Special roles
)
stxn = txn.sign(MyAlgo['private'])             # Sign
txid = algod_client.send_transaction(stxn) 
txinfo = wait_for_confirmation(algod_client,txid)
# Get the asset ID and open in Algoexplorer
SSC_id = txinfo['asset-index'] #80679348 -> Asset index!
print(SSC_id)
print('https://testnet.algoexplorer.io/asset/{}'.format(SSC_id))

Current round is  20613951.
Waiting for round 20613951 to finish.
Waiting for round 20613952 to finish.
Transaction L5NTFNIYHRDUNNH4UKNNH3CQBSQ5KV52UFJ25H5YYRHTNDH2MXWA confirmed in round 20613953.
80684006
https://testnet.algoexplorer.io/asset/80684006


### Create a dispenser for the Social Coin and fund it

In [457]:
import random
from random import randrange

a = Int( randrange( (2**32-1) ))
random_cond = ( a == a )
fee_condition =  (Txn.fee() <= Int(1000))            # Max fee is 1000micro Algos (0.001 Algos)
other_condition = (Txn.asset_amount() <= Int(int(1e2)))   # Everyone should only be able to request less or equal 1 Social Coin
safety_condition = And (
        Txn.type_enum() == TxnType.AssetTransfer,          # Must be a "Assetpayment" transaction
        Txn.rekey_to() == Global.zero_address(),      # Cannot change private key
        Txn.xfer_asset() == Int(SSC_id),             # must be Social Coin                 
)

sc_dispenser_pyteal = And(
    random_cond,fee_condition,safety_condition,other_condition
)
sc_dispenser_teal = compileTeal(sc_dispenser_pyteal, 
                             Mode.Signature,          # <----- Here we say it is a Smart Signature (and not a Smart Contract)
                             version=3)
print(sc_dispenser_teal)

sc_Dispenser = algod_client.compile(sc_dispenser_teal)

print('http://testnet.algoexplorer.io/address/'+sc_Dispenser['hash'])

#pragma version 3
int 926470377
int 926470377
==
txn Fee
int 1000
<=
&&
txn TypeEnum
int axfer
==
txn RekeyTo
global ZeroAddress
==
&&
txn XferAsset
int 80684006
==
&&
&&
txn AssetAmount
int 100
<=
&&
return
http://testnet.algoexplorer.io/address/ZOAMAWCZKB6HTGEQS4VRYA2YYZAA636YZYCK3CAKWL4QYNVTTGRMCL7FLY


In [458]:
# Now we fund it first with Algo
sp = algod_client.suggested_params()
amt = int(1*1e6)
txn = PaymentTxn(sender=MyAlgo['public'], sp=sp, receiver=sc_Dispenser['hash'], amt=amt )

# Step 2+3: sign and sen
stxn = txn.sign(MyAlgo['private'])
txid = algod_client.send_transaction(stxn)

# Step 4: wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)


Current round is  20614022.
Waiting for round 20614022 to finish.
Waiting for round 20614023 to finish.
Transaction VAPRY53RUDSU7XCKY4EYNF6EZNQHBPPZUUHYTJZUUG4FIQOKP43Q confirmed in round 20614024.


In [459]:
# First, the dispenser needs to opt-in
# Steo 5.1: Prepare
sp = algod_client.suggested_params()
txn = AssetTransferTxn(sc_Dispenser['hash'], sp, sc_Dispenser['hash'], 0, SSC_id) # 0 Social Coin Transfer for the opt-in

# Steo 5.2: Sign
encodedProg = sc_Dispenser['result'].encode()
program = base64.decodebytes(encodedProg)
lsig = LogicSig(program)
stxn = LogicSigTransaction(txn, lsig)

# Step 5.3 Send
txid = algod_client.send_transaction(stxn)

# Step 5.4 Wait for ...
txinfo = wait_for_confirmation(algod_client, txid)

print('http://testnet.algoexplorer.io/tx/'+txid)

Current round is  20614024.
Waiting for round 20614024 to finish.
Waiting for round 20614025 to finish.
Transaction KGTHTSUIJQ5ZRLBBOUTG7QALBOUYAC5O6VBPF3BTZ4QDI4KXTUVQ confirmed in round 20614026.
http://testnet.algoexplorer.io/tx/KGTHTSUIJQ5ZRLBBOUTG7QALBOUYAC5O6VBPF3BTZ4QDI4KXTUVQ


In [460]:
# Now we fund it with Social Coin
# Step 1: prepare transaction
sp = algod_client.suggested_params()
amt = int(500*1e2)
txn = AssetTransferTxn(sender=MyAlgo['public'], sp=sp, receiver=sc_Dispenser['hash'], amt=amt,index =SSC_id )

# Step 2+3: sign and send
stxn = txn.sign(MyAlgo['private'])
txid = algod_client.send_transaction(stxn)

# Step 4: wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  20614026.
Waiting for round 20614026 to finish.
Waiting for round 20614027 to finish.
Transaction PIN7F6XVCTVWH2PEV4OINWRCQUHTUMORUO5YPVBGIYGBYCTZ343A confirmed in round 20614028.


In [461]:
# Now Alice should own some Social Coin

# First, opt-in
sp = algod_client.suggested_params()

txn = AssetTransferTxn(sender=Alice['public'], sp=sp, receiver=Alice['public'], amt=0,index =SSC_id )

# Step 2+3: sign and send
stxn = txn.sign(Alice['private'])
txid = algod_client.send_transaction(stxn)

# Step 4: wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)


Current round is  20614028.
Waiting for round 20614028 to finish.
Waiting for round 20614029 to finish.
Transaction PFVLLOHF6NQ764CQFWV7QYW5RVPQJ3DPC2XG7SX5CTIUIICQWNPQ confirmed in round 20614030.


In [462]:
# Get Social Coin
# Step 1: prepare TX
sp = algod_client.suggested_params()
withdrawal_amt = int(1*1e2)
txn = AssetTransferTxn(sender=sc_Dispenser['hash'], sp=sp, receiver=Alice['public'], amt=withdrawal_amt, index = SSC_id)

# Step 2: sign TX 
encodedProg = sc_Dispenser['result'].encode() # result is basically the program code
program = base64.decodebytes(encodedProg)
lsig = LogicSig(program)
stxn = LogicSigTransaction(txn, lsig) # logic signature = smart signature

# Step 3: send
txid = algod_client.send_transaction(stxn)

# Step 4: wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  20614030.
Waiting for round 20614030 to finish.
Waiting for round 20614031 to finish.
Transaction R3PFGOPHPORT4DSITXFUXU6X73PACUI5DTT7XLBIDSQP3KAJYPXQ confirmed in round 20614032.


## Create an Algo dispenser with conditions

In [125]:
def read_local_state(client, addr, app_id):
    # reads a user's local state
    # client = algod_client
    # addr = public addr of the user that we want to inspect
    results = client.account_info(addr)
    for local_state in results["apps-local-state"]:
        if local_state["id"] == app_id:
            if "key-value" not in local_state:
                return {}
            return format_state(local_state["key-value"])
    return {}

def read_global_state(client, app_id):
    # reads an app's global state
    return  algod_client.application_info(app_id)["params"]["global-state"]

def format_state(state):
    # formats the state (local/global) nicely 
    formatted = {}
    textvariables = {'Info','Note'}        # <---- update this! (List of Text variables in SC)
    for item in state:
        key = base64.b64decode(item["key"]).decode("utf-8")
        value = item["value"]
        if value["type"] == 1:
            if key in textvariables:                 # Format text variables
                formatted_value = base64.b64decode(value["bytes"]).decode("utf-8")
            else:                                    # Format addresses
                formatted_value = base64.b32encode(base64.b64decode(value["bytes"]))
            formatted[key] = formatted_value
        else:
            formatted[key] = value["uint"]
    return formatted

In [417]:
handle_creation =  Return( Int(1) )               # Not doing anything, returning "OK"  
    
handle_optin =  Return( Int(1) )                  # Not doing anything, returning "OK"  

handle_closeout = Return( Int(1) )                # Not doing anything, returning "OK"  

handle_updateapp = Return( Int(0) )               # Always FALSE ... updating not allowed

handle_deleteapp = Return(
    Txn.sender() == Global.creator_address()    # only TRUE if delete request is made by creator
)
#handle interaction
socialcbalance = AssetHolding.balance(Txn.sender(),Txn.assets[0]) # preparing the social coin asset to use it in the NoOp transaction

handle_noop  = Seq([socialcbalance,Assert(socialcbalance.value() > Int(0)), # check whether the user owns any Social Coin
                    Assert(Balance(Txn.sender()) < Int(int(10*1e6))), # check whether the user holds less than 10 Algo
                    InnerTxnBuilder.Begin(), # initiate an inner transaction
                    InnerTxnBuilder.SetFields({ # set the necassary parameters
                    TxnField.type_enum: TxnType.Payment, # It should be a payment transaction (we want to send Algo)
                    TxnField.amount: Int(int(10*1e6))-Balance(Txn.sender()), # amount should be 10 Algo minus the balance of the user
                    TxnField.receiver: Txn.sender() # The user of the contract is the transaction receiver
                                    }),
                    InnerTxnBuilder.Submit(), # submit the transaction
                   Return (Int(1) )]) # return "OK"
                                    


     

In [418]:
algo_approval_pyteal = Cond( # Approval function
    [Txn.application_id() == Int(0),              handle_creation],
    [Txn.on_completion()  == OnComplete.OptIn,    handle_optin],
    [Txn.on_completion()  == OnComplete.CloseOut, handle_closeout],
    [Txn.on_completion()  == OnComplete.UpdateApplication, handle_updateapp],
    [Txn.on_completion()  == OnComplete.DeleteApplication, handle_deleteapp],
    [Txn.on_completion()  == OnComplete.NoOp, handle_noop],
)

In [419]:
algo_approval_teal = compileTeal(algo_approval_pyteal,mode=Mode.Application, version=5) # using version 5 to allow inner tx

In [420]:
algo_clear_pyteal =  Return( Int(1) )  # clear function
algo_clear_teal = compileTeal(algo_clear_pyteal,mode=Mode.Application, version=5) # compile to teal

algo_approval_b64 = algod_client.compile(algo_approval_teal) 
algo_Approval =  base64.b64decode(algo_approval_b64['result'])

algo_clear_b64 = algod_client.compile(algo_clear_teal)
algo_Clear =  base64.b64decode(algo_clear_b64['result'])

In [421]:
# Step 1: Prepare the transaction
sp = algod_client.suggested_params()

# How much space do we need? -> none
global_ints = 0    # no numeric variable
global_bytes = 0   # 
algo_global_schema = transaction.StateSchema(global_ints, global_bytes)

local_ints = 0     # No local variables
local_bytes = 0    # 
algo_local_schema = transaction.StateSchema(local_ints, local_bytes)

txn = transaction.ApplicationCreateTxn(
      sender = MyAlgo['public'],              # <-- sender public
      sp = sp,                             # <-- sp
      on_complete = 0,                     # <-- when finished do nothing
      approval_program = algo_Approval,   # <-- approval program 
      clear_program = algo_Clear,         # <-- clear program 
      global_schema = algo_global_schema, # <-- reserve global space 
      local_schema = algo_local_schema,    # <-- reserve local space
      foreign_assets = [SSC_id]           # <-- include foreign-asset 'Social Coin', to use it in the NoOp transaction
    )

# Step 2: sign transaction
stxn = txn.sign(MyAlgo['private'])

# Step 3: send
txid=algod_client.send_transactions([stxn])

# Step 4: wait for ...
txinfo = wait_for_confirmation(algod_client, txid)
app_id = txinfo["application-index"]
print("Created new app-id:", app_id)
print('https://testnet.algoexplorer.io/application/{}'.format(app_id))

Current round is  20613442.
Waiting for round 20613442 to finish.
Waiting for round 20613443 to finish.
Transaction 75QM2CL2PJYLVBVRQBCM2WUUSDDTJLNTPC3VSCC5IPQEHELIH74A confirmed in round 20613444.
Created new app-id: 80680577
https://testnet.algoexplorer.io/application/80680577


In [422]:
app_id2 = int(app_id) # get app_id as integer
app_addr = logic.get_application_address(app_id2) # get the public address of the smart contract
print(app_addr)

6UNI5OCIXXVIISQ77IFC7ZWLH4OGNOS4WSSSCKWW5F7AVXF5F3L4HLPCKY


## Sending some Algo to fund the Smart Contract
### This step is not necassary if funding is done by a dispenser!

In [423]:
sp       = algod_client.suggested_params()       # suggested params
amount   = 3
algo_precision = 1e6
amt_microalgo = int(amount * algo_precision)

txn = PaymentTxn(sender = MyAlgo['public'],     # <--- From
                 sp = sp, 
                 receiver = app_addr,      # <---- To
                 amt = amt_microalgo            # <---- Amount in Micro-ALGOs
                )

# Step 2: sign
stxn = txn.sign(MyAlgo['private']) 

# Step 3: send
txid = algod_client.send_transaction(stxn)

# Step 4: Wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  20613444.
Waiting for round 20613444 to finish.
Waiting for round 20613445 to finish.
Transaction WMS7XCAL5NO33BBJ5KQLH6TUVMJ7XI23LLHWS5N4KQIVADRUDX2A confirmed in round 20613446.


### Let the users opt-in to the smart contract

In [424]:
users = [Alice,Bob,Charlie,Dina,MyAlgo]

for user in users: # iterate for every user
# Step 1: prepare transaction
    sp = algod_client.suggested_params()
    txn = transaction.ApplicationOptInTxn(user['public'], sp, app_id,foreign_assets=[SSC_id])

# Step 2: sign transaction
    stxn = txn.sign(user['private'])

# Step 3: send
    txid = algod_client.send_transactions([stxn])

# Step 4: await confirmation
    txinfo = wait_for_confirmation(algod_client, txid)

Current round is  20613446.
Waiting for round 20613446 to finish.
Waiting for round 20613447 to finish.
Transaction XNAFWZMXVGU3K4HRAJPJBGMUMTTW73L2VOOGDZYPZY725GLTTBBA confirmed in round 20613448.
Current round is  20613448.
Waiting for round 20613448 to finish.
Waiting for round 20613449 to finish.
Transaction JHC25VF2Q5Y6422M36XNOXALCL2XXZXJHDL2ZWTRCPDR7Y6FO2KA confirmed in round 20613450.
Current round is  20613450.
Waiting for round 20613450 to finish.
Waiting for round 20613451 to finish.
Transaction TPRR3D2LWQRWEA5LXXCVO4NR2ITZQII522SV4QWNQHPQRTT3NX2Q confirmed in round 20613452.
Current round is  20613452.
Waiting for round 20613452 to finish.
Waiting for round 20613453 to finish.
Transaction IKLJ6NYVMMDSQ6CQS63BFIPLFVEM5XKACQS44GCYIZC4Z4HNTPSQ confirmed in round 20613454.
Current round is  20613454.
Waiting for round 20613454 to finish.
Waiting for round 20613455 to finish.
Transaction 6WSMAGTI3YVCVEZLKMEWXRSXQYCM7OAYGWN5LB2SRFRCZYEC6BGQ confirmed in round 20613456.
